In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import jieba

## read random select review csv

In [2]:
df = pd.read_csv("csv/random_review_text_restaurant.csv",header=None)
df.loc[:,1]

0                     空調位置不好,較悶熱。假日人多時，上菜速度緩慢,一頓飯超過2小時。  
1       中午約莫一點在店內待了至少十分鐘以上，櫃台完全沒人，也稍微喊了一下也無人回應，請問店是有在開...
2       等了半小時說單子不見，還完全不記得我們點什麼，完全沒開始做餐，要我們重劃單，但錢都收了，竟然...
3       今天聽到一位附近工地客人跟店家要隔熱套，店家說沒有喔！ 隨客人離開後，店家居然說，手繭這麼厚...
4       22/05/13:   不好意思，這裡的徒手包含沒有戴手套喔，若是有戴手套我也不會特地來留言...
                              ...                        
1495    菜色很多，又便宜，還有免費的甜湯跟湯品，在這個精華地段，有這樣CP值高的自助餐，真是附近居民...
1496    目前好像只有這裡的豆漿，喝得出是自己用鐵鍋煮的那種自然焦味。      很多早餐店的豆漿是用...
1497                 好吃！   肉好大塊   麵好Q彈   吃得很飽   這CP值頗高的  
1498    第一次吃到這種口感的蛋餅，很好吃也不用等很久   外皮薄脆香，有點像在吃蘇打餅乾的感覺   ...
1499                           超讚   老闆再次新發明   千層蛋佐暖心醬油糕  
Name: 1, Length: 1500, dtype: object

## tokenize with jieba

In [3]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file,encoding="utf-8") as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

In [4]:
stop_words_file = "stopwords.txt"
stopwords = get_custom_stopwords(stop_words_file)

In [5]:
poun=('，','。','！','：','「','」','…','、','？','【','】','.',':','?',';','!','~','`','+','-','<','>','/','[',']','{','}',"'",'"', '\n'," ","")
list_ = []
jieba.set_dictionary('dict.txt.big')
for originReview in df.loc[:,1]:
    # 刪除標點符號
    filteredText = "".join(c for c in originReview if c not in (poun))
    # 讀入停用詞
    stopWords = {}.fromkeys([ line.rstrip() for line in stopwords])
    #  以精確模式分詞
    words = jieba.cut(filteredText, cut_all=False)
    # 去除停用詞後, 以空白斷詞
    final = ''
    for seg in words:
        if seg not in stopWords:
            seg+=' '
            final += seg
    list_.append(final)
df['cutted_comment'] = list_

Building prefix dict from C:\Users\mason89823\BDSE28_FinalTopic\個人GIT\dict.txt.big ...
Loading model from cache C:\Users\MASON8~1\AppData\Local\Temp\jieba.ub2ca4d1fa2c360f5b7b131e89caf64f2.cache
Loading model cost 0.866 seconds.
Prefix dict has been built successfully.


In [6]:
df.loc[:,"cutted_comment"]

0                  空調 位置 不好 悶熱 假日 人多時 上菜 速度 緩慢 一頓飯 超過 小時 
1       中午 約莫 一點 店 內待 至少 十分鐘 以上 櫃台 完全 沒人 稍微 喊 一下 無人 回應...
2       半小 時說 單子 不見 完全 記得 點 完全 做餐 重劃 錢 收 完全 明細 紀錄 外 送 ...
3       今天 聽到 一位 附近 工地 客人 店家 隔熱 套 店家 說 喔 客人 離開 後 店家 說手...
4       220513 不好意思 徒手 包含 戴 手套 喔 戴 手套 特地來 留言 謝謝 想給 建議 ...
                              ...                        
1495          菜色 便宜 免費 甜湯 湯品 精華 地段 CP 值高 自助餐 真是 附近 居民 福氣 
1496    目前 好像 豆漿 喝 得出 鐵鍋 煮 那種 自然 焦味 早餐 店 豆漿 封裝 好 豆漿 袋 ...
1497                       好吃 肉 好 大塊 麵 好 Q 彈吃得 很飽 CP 值 高 
1498    第一次 吃 這種 口感 蛋餅 好吃 不用 久 外皮 薄脆 香 有點像 吃 蘇打餅乾 感覺 口...
1499                          超 讚 老闆 再次 新發明 千層 蛋佐暖心 醬油 糕 
Name: cutted_comment, Length: 1500, dtype: object

In [7]:
# 1 = [0,0,0]
# 2 = [0,0,1]
# 3 = [0,1,0]
# 4 = [1,0,0]
# 5 = [0,1,1]
df.loc[:,"new_rating"] = df.loc[:,0].apply(lambda x : [0,0,0] if x == 1 else ([0,0,1] if x == 2 else ([0,1,0] if x ==3 else ([1,0,0] if x == 4 else [0,1,1]))))

In [8]:
df.head()

,0,1,cutted_comment,new_rating
0,1,"空調位置不好,較悶熱。假日人多時，上菜速度緩慢,一頓飯超過2小時。",空調 位置 不好 悶熱 假日 人多時 上菜 速度 緩慢 一頓飯 超過 小時,"[0, 0, 0]"
1,1,中午約莫一點在店內待了至少十分鐘以上，櫃台完全沒人，也稍微喊了一下也無人回應，請問店是有在開...,中午 約莫 一點 店 內待 至少 十分鐘 以上 櫃台 完全 沒人 稍微 喊 一下 無人 回應...,"[0, 0, 0]"
2,1,等了半小時說單子不見，還完全不記得我們點什麼，完全沒開始做餐，要我們重劃單，但錢都收了，竟然...,半小 時說 單子 不見 完全 記得 點 完全 做餐 重劃 錢 收 完全 明細 紀錄 外 送 ...,"[0, 0, 0]"
3,1,今天聽到一位附近工地客人跟店家要隔熱套，店家說沒有喔！ 隨客人離開後，店家居然說，手繭這麼厚...,今天 聽到 一位 附近 工地 客人 店家 隔熱 套 店家 說 喔 客人 離開 後 店家 說手...,"[0, 0, 0]"
4,1,22/05/13: 不好意思，這裡的徒手包含沒有戴手套喔，若是有戴手套我也不會特地來留言...,220513 不好意思 徒手 包含 戴 手套 喔 戴 手套 特地來 留言 謝謝 想給 建議 ...,"[0, 0, 0]"


In [25]:
# define documents
X = df.loc[:,"cutted_comment"]

# define class labels
y = np.array(df.loc[:,"new_rating"].to_list())

vocab_size = 1500
maxlen = 300
encoded_docs = [one_hot(d, vocab_size) for d in X]
padded_docs = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')

model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, 8, input_length=maxlen))
model.add(layers.Flatten())

# 加上一般的完全連接層(Dense)
model.add(layers.Dense(3, activation='sigmoid'))
model.add(layers.Dense(3, activation='sigmoid'))
model.add(layers.Dense(3, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# summarize the model
print(model.summary())

# fit the model
X_train, X_test, y_train, y_test = train_test_split(padded_docs, y, random_state=1,test_size=0.2)
model.fit(X_train, y_train,epochs=200)

# evaluate the model
loss, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %f' % (accuracy*100))


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 300, 8)            12000     
                                                                 
 flatten_14 (Flatten)        (None, 2400)              0         
                                                                 
 dense_48 (Dense)            (None, 3)                 7203      
                                                                 
 dense_49 (Dense)            (None, 3)                 12        
                                                                 
 dense_50 (Dense)            (None, 3)                 12        
                                                                 
Total params: 19,227
Trainable params: 19,227
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
38/38 [=========================

38/38 [==============================] - 0s 1ms/step - loss: 0.1301 - accuracy: 0.5992
Epoch 73/200
38/38 [==============================] - 0s 1ms/step - loss: 0.1296 - accuracy: 0.5992
Epoch 74/200
38/38 [==============================] - 0s 1ms/step - loss: 0.1292 - accuracy: 0.5983
Epoch 75/200
38/38 [==============================] - 0s 1ms/step - loss: 0.1287 - accuracy: 0.5992
Epoch 76/200
38/38 [==============================] - 0s 1ms/step - loss: 0.1282 - accuracy: 0.5992
Epoch 77/200
38/38 [==============================] - 0s 1ms/step - loss: 0.1277 - accuracy: 0.5992
Epoch 78/200
38/38 [==============================] - 0s 1ms/step - loss: 0.1271 - accuracy: 0.5983
Epoch 79/200
38/38 [==============================] - 0s 1ms/step - loss: 0.1264 - accuracy: 0.5992
Epoch 80/200
38/38 [==============================] - 0s 1ms/step - loss: 0.1256 - accuracy: 0.6150
Epoch 81/200
38/38 [==============================] - 0s 1ms/step - loss: 0.1248 - accuracy: 0.7525
Epoch 82/200


38/38 [==============================] - 0s 1ms/step - loss: 0.0518 - accuracy: 0.9850
Epoch 154/200
38/38 [==============================] - 0s 2ms/step - loss: 0.0510 - accuracy: 0.9842
Epoch 155/200
38/38 [==============================] - 0s 2ms/step - loss: 0.0501 - accuracy: 0.9867
Epoch 156/200
38/38 [==============================] - 0s 1ms/step - loss: 0.0493 - accuracy: 0.9892
Epoch 157/200
38/38 [==============================] - 0s 1ms/step - loss: 0.0485 - accuracy: 0.9700
Epoch 158/200
38/38 [==============================] - 0s 1ms/step - loss: 0.0478 - accuracy: 0.9867
Epoch 159/200
38/38 [==============================] - 0s 1ms/step - loss: 0.0471 - accuracy: 0.9875
Epoch 160/200
38/38 [==============================] - 0s 1ms/step - loss: 0.0463 - accuracy: 0.9933
Epoch 161/200
38/38 [==============================] - 0s 1ms/step - loss: 0.0456 - accuracy: 0.9958
Epoch 162/200
38/38 [==============================] - 0s 1ms/step - loss: 0.0448 - accuracy: 0.9967
Epoc

In [30]:
# define documents
X = df.loc[:,"cutted_comment"]

# define class labels
y = np.array(df.loc[:,"new_rating"].to_list())

vocab_size = 1500
maxlen = 300
encoded_docs = [one_hot(d, vocab_size) for d in X]
padded_docs = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')

model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, 8, input_length=maxlen))
model.add(layers.SimpleRNN(128))

# 加上一般的完全連接層(Dense)
model.add(layers.Dense(3, activation='sigmoid'))
model.add(layers.Dense(3, activation='sigmoid'))
model.add(layers.Dense(3, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# summarize the model
print(model.summary())

# fit the model
X_train, X_test, y_train, y_test = train_test_split(padded_docs, y, random_state=1,test_size=0.2)
model.fit(X_train, y_train,epochs=200)

# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %f' % (accuracy*100))


Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 300, 8)            12000     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 128)               17536     
                                                                 
 dense_63 (Dense)            (None, 3)                 387       
                                                                 
 dense_64 (Dense)            (None, 3)                 12        
                                                                 
 dense_65 (Dense)            (None, 3)                 12        
                                                                 
Total params: 29,947
Trainable params: 29,947
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
38/38 [=========================

38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 72/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 73/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 74/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.2675
Epoch 75/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 76/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 77/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 78/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 79/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 80/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epo

38/38 [==============================] - 1s 31ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 152/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 153/200
38/38 [==============================] - 1s 31ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 154/200
38/38 [==============================] - 1s 31ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 155/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 156/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 157/200
38/38 [==============================] - 1s 32ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 158/200
38/38 [==============================] - 1s 31ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 159/200
38/38 [==============================] - 1s 31ms/step - loss: 0.2137 - accuracy: 0.1992
Epoch 160/200
38/38 [==============================] - 1s 31ms/step - loss: 0.2138 - accuracy: 0